# K-Fold를 사용하여 모델 생성

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import joblib

# 맥에서 글씨 깨짐 방지
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [14]:
df = pd.read_csv('../data/27_school_district.csv')

/tmp/ipykernel_459681/296727937.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/27_school_district.csv')


In [15]:
df.shape

(1128094, 103)

In [18]:
# 일부 feature 제거 - 기준은 내마음대로

df.drop(['해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호',
         'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
         'k-사용검사일-사용승인일', 'k-관리비부과면적', 'k-전용면적별세대현황(60이하)', 'k-전용면적별세대현황(60~85이하)',
         'k-85~135이하', 'k-135초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태',
         '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일'], axis=1, inplace=True)

KeyError: "['해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-사용검사일-사용승인일', 'k-관리비부과면적', 'k-전용면적별세대현황(60이하)', 'k-전용면적별세대현황(60~85이하)', 'k-85~135이하', 'k-135초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일'] not found in axis"

In [17]:
df.shape

(1128094, 74)

In [7]:
df.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도',
       '도로명', 'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', '세대전기계약방법',
       '청소비관리형태', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', 'is_test', '시군구 번지',
       '좌표X,좌표Y', '가장 가까운 거리', 'index', '역사_ID', '역사명', '호선', '위도', '경도',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '가장 가까운 버스 정류장 index',
       '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명',
       '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표',
       '인근 버스 정류장 개수', '계약년', '계약월', 'GDP', '한국은행 기준금리', '기대 인플레이션', '지가지수',
       '아파트 인허가', '미분양', '거래량', '건설사 랭킹', '구', '동', '구별 지가지수', '공시지가 평균',
       '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수', '매매가격 지수 증감률',
       '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리', '건설사 카테고리',
       '가장 가까운 다리와의 거리', '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수',
       '인근 다리 개수 1개이상 3개 이하', '학군'],
      dtype='object')

In [19]:
# 추가로 제거가 필요한 feature 제거
df.drop(['시군구', '번지', '본번', '부번', '아파트명', '도로명', 'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적',
         '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '시군구 번지', '좌표X,좌표Y', 'index', '역사_ID', '역사명', '위도', '경도',
         '가장 가까운 버스 정류장 index', '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명',
         '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표', '동', 
         '가장 가까운 다리 index','가장 가까운 다리', '구', '호선'], axis=1, inplace=True)

In [20]:
df.columns

Index(['전용면적', '계약년월', '계약일', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test',
       '가장 가까운 거리', '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년',
       '계약월', 'GDP', '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량',
       '건설사 랭킹', '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수',
       '거래활발지수', '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리',
       '구 카테고리', '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수',
       '인근 다리 개수 1개이상 3개 이하', '학군'],
      dtype='object')

In [21]:
df.shape

(1128094, 40)

In [22]:
print(df.shape)
df.head(10)

(1128094, 40)


,전용면적,계약년월,계약일,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군
0,79.97,201712,8,3,1987,127.05721,37.476763,124000.0,0,1127.738351,...,1.243615,70.0590,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
1,79.97,201712,22,4,1987,127.05721,37.476763,123500.0,0,1127.738351,...,1.243615,70.0590,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
2,54.98,201712,28,5,1987,127.05721,37.476763,91500.0,0,1127.738351,...,1.243615,70.0590,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
3,79.97,201801,3,4,1987,127.05721,37.476763,130000.0,0,1127.738351,...,2.459291,69.3220,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
4,79.97,201801,8,2,1987,127.05721,37.476763,117000.0,0,1127.738351,...,2.459291,69.3220,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
5,79.97,201801,11,1,1987,127.05721,37.476763,130000.0,0,1127.738351,...,2.459291,69.3220,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
6,79.97,201803,19,2,1987,127.05721,37.476763,139500.0,0,1127.738351,...,2.180755,67.1875,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
7,54.98,201804,5,5,1987,127.05721,37.476763,107500.0,0,1127.738351,...,1.280568,66.2450,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
8,79.97,201806,28,3,1987,127.05721,37.476763,145000.0,0,1127.738351,...,0.274474,65.3984,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
9,54.98,201807,9,3,1987,127.05721,37.476763,112000.0,0,1127.738351,...,0.218161,64.8121,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0


In [23]:
# 계약월 변수 생성

df['계약월'] = df['계약년월'].astype(str).str[4:].astype(int)
df

,전용면적,계약년월,계약일,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군
0,79.97,201712,8,3,1987,127.05721,37.476763,124000.0,0,1127.738351,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
1,79.97,201712,22,4,1987,127.05721,37.476763,123500.0,0,1127.738351,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
2,54.98,201712,28,5,1987,127.05721,37.476763,91500.0,0,1127.738351,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
3,79.97,201801,3,4,1987,127.05721,37.476763,130000.0,0,1127.738351,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
4,79.97,201801,8,2,1987,127.05721,37.476763,117000.0,0,1127.738351,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,202307,19,13,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128090,84.62,202307,25,12,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128091,101.65,202308,27,12,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.607676,50.991003,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128092,84.94,202309,2,18,2014,127.10672,37.618870,NaN,1,732.482898,...,-0.225827,51.082553,기타,기타,기타,기타,5918.624352,0.0,0,1.0


In [24]:
# 계약년월, 계약일, 건축년도, 계약년 변수 제거

df.drop(['계약년월', '계약일'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군
0,79.97,3,1987,127.05721,37.476763,124000.0,0,1127.738351,2.0,61.89584,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
1,79.97,4,1987,127.05721,37.476763,123500.0,0,1127.738351,2.0,61.89584,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
2,54.98,5,1987,127.05721,37.476763,91500.0,0,1127.738351,2.0,61.89584,...,1.243615,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
3,79.97,4,1987,127.05721,37.476763,130000.0,0,1127.738351,2.0,61.89584,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
4,79.97,2,1987,127.05721,37.476763,117000.0,0,1127.738351,2.0,61.89584,...,2.459291,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,13,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128090,84.62,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.641286,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128091,101.65,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.607676,50.991003,기타,기타,기타,기타,5918.624352,0.0,0,1.0
1128092,84.94,18,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,-0.225827,51.082553,기타,기타,기타,기타,5918.624352,0.0,0,1.0


In [25]:
df['계약월_sin'] = np.sin(2 * np.pi * df['계약월'] / 12)

In [26]:
df.drop(['계약월'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,계약월_sin
0,79.97,3,1987,127.05721,37.476763,124000.0,0,1127.738351,2.0,61.89584,...,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,-2.449294e-16
1,79.97,4,1987,127.05721,37.476763,123500.0,0,1127.738351,2.0,61.89584,...,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,-2.449294e-16
2,54.98,5,1987,127.05721,37.476763,91500.0,0,1127.738351,2.0,61.89584,...,70.059000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,-2.449294e-16
3,79.97,4,1987,127.05721,37.476763,130000.0,0,1127.738351,2.0,61.89584,...,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,5.000000e-01
4,79.97,2,1987,127.05721,37.476763,117000.0,0,1127.738351,2.0,61.89584,...,69.322000,기타,분당선,강남구,기타,5581.622466,0.0,0,8.0,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,13,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-5.000000e-01
1128090,84.62,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,50.938744,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-5.000000e-01
1128091,101.65,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,50.991003,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-8.660254e-01
1128092,84.94,18,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,51.082553,기타,기타,기타,기타,5918.624352,0.0,0,1.0,-1.000000e+00


In [27]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

In [28]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

## 스케일링

In [29]:
def scailing(col, scaler_type):
    if scaler_type == 'min_max': scaler = MinMaxScaler()
    else : scaler = StandardScaler()

    return scaler.fit_transform(df[[col]])

cols = ['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율',
       '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하',
       '계약월_sin']

for col in cols:
    print(col)
    scaler_type = 'min_max'
    if col == '계약월_sin': scaler_type = 'standard'

    df[[col]] = scailing(col, scaler_type)

전용면적
층
건축년도
좌표X
좌표Y
가장 가까운 거리
인근 지하철 역 개수
가장 가까운 버스 정류장 거리
인근 버스 정류장 개수
계약년
GDP
한국은행 기준금리
기대 인플레이션
지가지수
아파트 인허가
미분양
거래량
구별 지가지수
공시지가 평균
매수우위지수
건물나이
구매력지수
거래활발지수
매매가격 지수 증감률
매매 대비 전세가격 비율
가장 가까운 다리와의 거리
인근 한강다리 개수
인근 다리 개수 1개이상 3개 이하
계약월_sin


## 호선 one-hot encoding

In [19]:
ohe = OneHotEncoder(sparse=False)
ohe_re = ohe.fit_transform(df[['호선']])

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
df = pd.concat([df, pd.DataFrame(ohe_re, columns=[col for col in ohe.categories_[0]])], axis=1)

In [21]:
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,호선,인근 지하철 역 개수,...,경춘선,공항철도1호선,과천선,분당선,신림선,신분당선,신분당선(연장2),우이신설선,일산선,중앙선
0,0.168839,0.095890,0.419355,0.678243,0.167126,124000.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.168839,0.109589,0.419355,0.678243,0.167126,123500.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.108520,0.123288,0.419355,0.678243,0.167126,91500.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.168839,0.109589,0.419355,0.678243,0.167126,130000.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.168839,0.082192,0.419355,0.678243,0.167126,117000.0,0,0.353721,분당선,0.086957,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,0.180135,0.232877,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128090,0.180063,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128091,0.221168,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128092,0.180835,0.301370,0.854839,0.807949,0.728162,NaN,1,0.227756,경춘선,0.173913,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df.drop(['호선'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,경춘선,공항철도1호선,과천선,분당선,신림선,신분당선,신분당선(연장2),우이신설선,일산선,중앙선
0,0.168839,0.095890,0.419355,0.678243,0.167126,124000.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.168839,0.109589,0.419355,0.678243,0.167126,123500.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.108520,0.123288,0.419355,0.678243,0.167126,91500.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.168839,0.109589,0.419355,0.678243,0.167126,130000.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.168839,0.082192,0.419355,0.678243,0.167126,117000.0,0,0.353721,0.086957,0.101599,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,0.180135,0.232877,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128090,0.180063,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128091,0.221168,0.219178,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128092,0.180835,0.301370,0.854839,0.807949,0.728162,NaN,1,0.227756,0.173913,0.097533,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '거래량', '구', '구별 지가지수', '공시지가 평균', '매수우위지수',
       '30년이상50년이하', '계약월_sin', '1호선', '2호선', '3호선', '4호선', '5호선', '6호선',
       '7호선', '8호선', '9호선', '9호선(연장)', '경부선', '경원선', '경의중앙선', '경인선', '경춘선',
       '공항철도1호선', '과천선', '분당선', '신림선', '신분당선', '신분당선(연장2)', '우이신설선', '일산선',
       '중앙선'],
      dtype='object')

In [24]:
df.drop(['9호선(연장)', '경부선', '8호선', 
         '경의중앙선', '신림선', '신분당선',
         '신분당선(연장2)', '중앙선', '우이신설선',
         '공항철도1호선', '경인선', '1호선',
         '일산선', '과천선', '경춘선'], axis=1, inplace=True)

In [30]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

# 카테고리 변수 처리
- 30년이상50년이하
- 아파트 카테고리
- 지하철 카테고리
- 구 카테고리
- 건설사 카테고리
- 인근 다리 개수 1개이상 3개 이하
- 학군

In [31]:
df.head()

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,계약월_sin
0,0.168839,0.095890,0.419355,0.678243,0.167126,124000.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
1,0.168839,0.109589,0.419355,0.678243,0.167126,123500.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
2,0.108520,0.123288,0.419355,0.678243,0.167126,91500.0,0,0.353721,0.086957,0.101599,...,0.864178,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,-0.022272
3,0.168839,0.109589,0.419355,0.678243,0.167126,130000.0,0,0.353721,0.086957,0.101599,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200
4,0.168839,0.082192,0.419355,0.678243,0.167126,117000.0,0,0.353721,0.086957,0.101599,...,0.844200,기타,분당선,강남구,기타,0.347392,0.0,0.0,8.0,0.684200


In [32]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

In [33]:
df['30년이상50년이하'] = df['30년이상50년이하'].astype('category')
df['아파트 카테고리'] = df['아파트 카테고리'].astype('category')
df['지하철 카테고리'] = df['지하철 카테고리'].astype('category')
df['구 카테고리'] = df['구 카테고리'].astype('category')
df['건설사 카테고리'] = df['건설사 카테고리'].astype('category')
df['인근 다리 개수 1개이상 3개 이하'] = df['인근 다리 개수 1개이상 3개 이하'].astype('category')
df['학군'] = df['학군'].astype('category')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 38 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   전용면적                 1128094 non-null  float64 
 1   층                    1128094 non-null  float64 
 2   건축년도                 1128094 non-null  float64 
 3   좌표X                  1128094 non-null  float64 
 4   좌표Y                  1128094 non-null  float64 
 5   target               1118822 non-null  float64 
 6   is_test              1128094 non-null  int64   
 7   가장 가까운 거리            1128094 non-null  float64 
 8   인근 지하철 역 개수          1128094 non-null  float64 
 9   가장 가까운 버스 정류장 거리     1128094 non-null  float64 
 10  인근 버스 정류장 개수         1128094 non-null  float64 
 11  계약년                  1128094 non-null  float64 
 12  GDP                  1128094 non-null  float64 
 13  한국은행 기준금리            1128094 non-null  float64 
 14  기대 인플레이션             1128094 non-n

# Dataset 생성

In [35]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군',
       '계약월_sin'],
      dtype='object')

In [36]:
# 추가한 feature들 중 일부 제거
df_dropped = df.drop(['지가지수', '건설사 랭킹', '인근 다리 개수 1개이상 3개 이하', '30년이상50년이하'], axis=1)

In [37]:
train_df = df_dropped[df_dropped['is_test'] == 0]
test_df = df_dropped[df_dropped['is_test'] == 1]

In [39]:
print(train_df.shape)
print(test_df.shape)

(1118822, 34)
(9272, 34)


In [40]:
train_df.drop(['is_test'], axis=1, inplace=True)
test_df.drop(['is_test'], axis=1, inplace=True)

/tmp/ipykernel_459681/1571330888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(['is_test'], axis=1, inplace=True)
/tmp/ipykernel_459681/1571330888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['is_test'], axis=1, inplace=True)


In [41]:
print(train_df.shape)
print(test_df.shape)

(1118822, 33)
(9272, 33)


In [42]:
train_X = train_df.drop(['target'], axis=1)
train_y = train_df[['target']]

In [43]:
train_X

,전용면적,층,건축년도,좌표X,좌표Y,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,인근 버스 정류장 개수,계약년,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,학군,계약월_sin
0,0.168839,0.095890,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6250,...,0.526489,0.864178,기타,분당선,강남구,기타,0.347392,0.0,8.0,-0.022272
1,0.168839,0.109589,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6250,...,0.526489,0.864178,기타,분당선,강남구,기타,0.347392,0.0,8.0,-0.022272
2,0.108520,0.123288,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6250,...,0.526489,0.864178,기타,분당선,강남구,기타,0.347392,0.0,8.0,-0.022272
3,0.168839,0.109589,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6875,...,0.639847,0.844200,기타,분당선,강남구,기타,0.347392,0.0,8.0,0.684200
4,0.168839,0.082192,0.419355,0.678243,0.167126,0.353721,0.086957,0.101599,0.200000,0.6875,...,0.639847,0.844200,기타,분당선,강남구,기타,0.347392,0.0,8.0,0.684200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,0.120492,0.205479,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.442831,0.131260,기타,6호선,기타,현대,0.383051,0.0,2.0,-0.728745
1118818,0.120492,0.191781,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.416283,0.127898,기타,6호선,기타,현대,0.383051,0.0,2.0,-1.245919
1118819,0.180570,0.328767,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.416283,0.127898,기타,6호선,기타,현대,0.383051,0.0,2.0,-1.245919
1118820,0.180570,0.164384,0.596774,0.281155,0.704836,0.326389,0.086957,0.131622,0.228571,0.0000,...,0.444495,0.125540,기타,6호선,기타,현대,0.383051,0.0,2.0,-1.435218


In [44]:
train_X.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '아파트 인허가', '미분양', '거래량', '구별 지가지수', '공시지가 평균', '매수우위지수',
       '건물나이', '구매력지수', '거래활발지수', '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리',
       '지하철 카테고리', '구 카테고리', '건설사 카테고리', '가장 가까운 다리와의 거리', '인근 한강다리 개수', '학군',
       '계약월_sin'],
      dtype='object')

In [45]:
train_y

,target
0,124000.0
1,123500.0
2,91500.0
3,130000.0
4,117000.0
...,...
1118817,20000.0
1118818,20000.0
1118819,28000.0
1118820,29000.0


# train_data shuffle

In [46]:
train_shuffled = train_df.sample(frac=1).reset_index(drop=True)
train_shuffled

,전용면적,층,건축년도,좌표X,좌표Y,target,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,인근 버스 정류장 개수,...,매매가격 지수 증감률,매매 대비 전세가격 비율,아파트 카테고리,지하철 카테고리,구 카테고리,건설사 카테고리,가장 가까운 다리와의 거리,인근 한강다리 개수,학군,계약월_sin
0,0.163505,0.068493,0.661290,0.284930,0.479681,37000.0,0.078244,0.217391,0.304467,0.400000,...,0.478018,0.961799,기타,6호선,기타,기타,0.058290,0.5,2.0,-1.245919
1,0.085759,0.136986,0.435484,0.685486,0.902305,23700.0,0.089144,0.217391,0.222762,0.457143,...,0.470613,0.961799,기타,7호선,기타,기타,0.792194,0.0,4.0,-1.245919
2,0.120492,0.068493,0.451613,0.807923,0.668654,26500.0,0.098895,0.173913,0.057683,0.314286,...,0.437280,0.895098,기타,기타,기타,기타,0.268368,0.0,1.0,-1.435218
3,0.180183,0.164384,0.709677,0.787298,0.678046,33000.0,0.249950,0.217391,0.158803,0.257143,...,0.404016,0.711305,기타,기타,기타,기타,0.308830,0.0,1.0,-1.245919
4,0.180850,0.136986,0.677419,0.295164,0.745322,46800.0,0.310244,0.130435,0.181642,0.057143,...,0.410523,0.571198,기타,6호선,기타,기타,0.463280,0.0,2.0,-0.022272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,0.221723,0.123288,0.854839,0.767289,0.183843,78000.0,0.208449,0.130435,0.116151,0.142857,...,0.444541,0.724206,포레,3호선,강남구,기타,0.297705,0.0,8.0,-1.245919
1118818,0.261236,0.136986,0.516129,0.464573,0.218180,56500.0,0.210313,0.173913,0.147397,0.314286,...,0.407441,0.351750,기타,4호선,기타,기타,0.131537,0.0,9.0,1.201374
1118819,0.114611,0.191781,0.677419,0.267158,0.595907,26800.0,0.201786,0.304348,0.091881,0.114286,...,0.479666,0.114453,기타,기타,기타,기타,0.203361,0.0,2.0,-0.728745
1118820,0.120541,0.164384,0.629032,0.599410,0.487460,36300.0,0.063798,0.565217,0.277609,0.514286,...,0.423500,0.179815,기타,5호선,성동구,기타,0.131227,0.0,10.0,0.684200


In [47]:
train_X = train_shuffled.drop(['target'], axis=1)
train_y = train_shuffled[['target']]

## K-Fold

In [48]:
kf = KFold(n_splits=10)

In [49]:
train_folds = kf.split(train_X, train_y)
display(train_folds)

<generator object _BaseKFold.split at 0x7f41be27b5a0>

In [50]:
params = {
    'n_estimators': 100000,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'metric_freq': 20,
    'device': 'gpu',
    'verbosity': 0
}

fold_models = []

for fold_idx, (train_idx, valid_idx) in enumerate(train_folds):
    print(f'--------------------{fold_idx}번째 학습 시작--------------------')

    train_X_fold = train_X.iloc[train_idx, :]
    train_y_fold = train_y.iloc[train_idx]

    valid_X_fold = train_X.iloc[valid_idx, :]
    valid_y_fold = train_y.iloc[valid_idx]

    model = lgb.LGBMRegressor(n_estimators=100000,
                              metric='rmse', data_sample_strategy='goss')
    
    model.fit(
    train_X_fold, train_y_fold,
    eval_set = [(train_X_fold, train_y_fold), (valid_X_fold, valid_y_fold)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=60),
               lgb.log_evaluation(period=10, show_stdv=True)])
    
    fold_models.append(model)
    joblib.dump(model, f'./16/kfold_models/model_{fold_idx}.pkl')
    print(f'--------------------{fold_idx}번째 학습 종료--------------------')

--------------------0번째 학습 시작--------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3821
[LightGBM] [Info] Number of data points in the train set: 1006939, number of used features: 32
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 57986.211912
Training until validation scores don't improve for 60 rounds
[10]	training's rmse: 26140.3	valid_1's rmse: 26425.2
[20]	training's rmse: 19064.8	valid_1's rmse: 19367.5
[30]	training's rmse: 15984.1	valid_1's rmse: 16340.8
[40]	training's rmse: 14490.1	valid_1's rmse: 14849.5
[50]	training's rmse: 13523.5	valid_1's rmse: 13890.9
[60]	training's rm

KeyboardInterrupt: 

# 예측

In [43]:
test_df.drop(['target'], axis=1, inplace=True)

/tmp/ipykernel_405396/1820151961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(['target'], axis=1, inplace=True)


In [44]:
preds_list = []

for model in fold_models:
    preds_list.append(model.predict(test_df))

In [45]:
preds_list[0]

array([166742.53062979, 308442.30353008, 311965.21271889, ...,
        85291.72538599,  74354.8030983 ,  72539.14838442])

In [47]:
pd.DataFrame(preds_list[0], columns=['target'])

,target
0,166742.530630
1,308442.303530
2,311965.212719
3,274091.610984
4,210712.203769
...,...
9267,71895.423778
9268,71618.972819
9269,85291.725386
9270,74354.803098


In [51]:
re_df = pd.concat([pd.DataFrame(preds, columns=['target_']) for preds in preds_list], axis=1)
re_df

,target_,target_,target_,target_,target_,target_,target_,target_,target_,target_
0,166742.530630,197438.452465,196425.139833,193125.339176,198640.543848,186993.024092,176926.820649,194586.050912,181234.645569,172384.411605
1,308442.303530,342522.796010,287018.264005,286751.722353,305626.829719,280544.648703,272194.563328,280710.729010,318813.653558,310983.134459
2,311965.212719,345689.876766,331083.235997,320112.854562,329007.827932,328430.166898,322480.240121,320117.431188,297990.664975,307332.905946
3,274091.610984,301726.233979,288701.518173,287842.070512,289974.174600,290847.388482,269677.385878,275220.349360,278885.847288,261700.638159
4,210712.203769,234758.662667,224520.501206,209418.217377,217973.898003,213547.095121,206539.166111,206951.216187,206571.455283,236830.400563
...,...,...,...,...,...,...,...,...,...,...
9267,71895.423778,73423.290167,78315.272695,69120.245114,75700.433914,72531.011434,74097.835631,68240.235407,65739.553525,67046.575104
9268,71618.972819,73547.977247,77828.678172,68257.246073,75850.195744,72345.545437,73580.731525,67352.077453,65829.677465,66981.801831
9269,85291.725386,91220.128019,89486.581389,83018.621890,88945.466850,85391.523310,85647.007337,84111.635540,81396.543621,88870.595682
9270,74354.803098,78370.063573,83139.761987,70675.904756,77123.628996,77211.004298,81446.760948,73183.038967,70033.375842,79211.054736


In [55]:
re_df['target'] = re_df.mean(axis=1)
re_df

,target_,target_,target_,target_,target_,target_,target_,target_,target_,target_,target
0,166742.530630,197438.452465,196425.139833,193125.339176,198640.543848,186993.024092,176926.820649,194586.050912,181234.645569,172384.411605,186449.695878
1,308442.303530,342522.796010,287018.264005,286751.722353,305626.829719,280544.648703,272194.563328,280710.729010,318813.653558,310983.134459,299360.864468
2,311965.212719,345689.876766,331083.235997,320112.854562,329007.827932,328430.166898,322480.240121,320117.431188,297990.664975,307332.905946,321421.041710
3,274091.610984,301726.233979,288701.518173,287842.070512,289974.174600,290847.388482,269677.385878,275220.349360,278885.847288,261700.638159,281866.721742
4,210712.203769,234758.662667,224520.501206,209418.217377,217973.898003,213547.095121,206539.166111,206951.216187,206571.455283,236830.400563,216782.281629
...,...,...,...,...,...,...,...,...,...,...,...
9267,71895.423778,73423.290167,78315.272695,69120.245114,75700.433914,72531.011434,74097.835631,68240.235407,65739.553525,67046.575104,71610.987677
9268,71618.972819,73547.977247,77828.678172,68257.246073,75850.195744,72345.545437,73580.731525,67352.077453,65829.677465,66981.801831,71319.290377
9269,85291.725386,91220.128019,89486.581389,83018.621890,88945.466850,85391.523310,85647.007337,84111.635540,81396.543621,88870.595682,86337.982902
9270,74354.803098,78370.063573,83139.761987,70675.904756,77123.628996,77211.004298,81446.760948,73183.038967,70033.375842,79211.054736,76474.939720


In [56]:
re_df = re_df[['target']]
re_df

,target
0,186449.695878
1,299360.864468
2,321421.041710
3,281866.721742
4,216782.281629
...,...
9267,71610.987677
9268,71319.290377
9269,86337.982902
9270,76474.939720


In [57]:
re_df = np.round(re_df).astype(int)
re_df

,target
0,186450
1,299361
2,321421
3,281867
4,216782
...,...
9267,71611
9268,71319
9269,86338
9270,76475


In [58]:
re_df.to_csv('../preds/v_16_kfold.csv', index=False)